# Demo collision detection 2D

In [ ]:
import torchlensmaker as tlm
import torch
import torch.nn
import math
from pprint import pprint

def make_uniform_rays(num_rays, start_x, end_x, max_y, dim):
    start_x = torch.full((num_rays,), start_x)
    start_y = torch.linspace(0., max_y, num_rays)
    
    rays_start = torch.stack((start_x, start_y), dim=1)

    rays_end = rays_start.clone()
    rays_end[:, 0] = end_x

    rays_vectors = torch.nn.functional.normalize(rays_end - rays_start, dim=1)

    return torch.hstack((rays_start, rays_vectors))

def make_random_rays(num_rays, start_x, end_x, max_y, dim):
    rays_start = (torch.rand((num_rays, dim)) * 2 - 1) * max_y
    rays_start[:, 0] = start_x

    rays_end = (torch.rand((num_rays, dim)) * 2 - 1) * max_y
    rays_end[:, 0] = end_x

    rays_vectors = torch.nn.functional.normalize(rays_end - rays_start, dim=1)

    return torch.hstack((rays_start, rays_vectors))


#test_rays = make_uniform_rays(num_rays=300, start_x=-10, end_x=40, max_y=13, dim=2)
test_rays = make_random_rays(num_rays=30, start_x=-20, end_x=50, max_y=15, dim=2)

Tensor = torch.Tensor


test_data = [
    (tlm.Surface2DTransform(1.0, "extent", 0., [35/2., -5]), tlm.surfaces.Sphere(35.0, 35/2)),
    (tlm.Surface2DTransform(-1.0, "extent", 0., [35/2., -5]), tlm.surfaces.Sphere(35.0, 35/2)),
    
    (tlm.Surface2DTransform(1.0, "origin", 0., [0, 0]), tlm.surfaces.Sphere(35.0, 35/2)),
    
    (tlm.Surface2DTransform(1.0, "origin", 5., [15., -5]), tlm.surfaces.Parabola(35.0, 0.010)),

    (tlm.Surface2DTransform(1.0, "origin", -15., [25., 12]), tlm.surfaces.Parabola(35.0, 0.010)),
    (tlm.Surface2DTransform(-1.0, "origin", -15., [25., 12]), tlm.surfaces.Parabola(35.0, 0.010)),

    (tlm.Surface2DTransform(1.0, "extent", 15., [40., 12]), tlm.surfaces.Parabola(35.0, 0.010)),
    (tlm.Surface2DTransform(-1.0, "extent", 15., [40., 12]), tlm.surfaces.Parabola(35.0, 0.010)),
    (tlm.Surface2DTransform(1.0, "origin", 0., [40., 0]), tlm.surfaces.Parabola(35.0, 0.010)),

    (tlm.Surface2DTransform(1.0, "origin", 0, [-5., 0]), tlm.surfaces.SquarePlane(35.0)),
    (tlm.Surface2DTransform(1.0, "origin", -40, [50., 0]), tlm.surfaces.CircularPlane(35.0)),
    (tlm.Surface2DTransform(1.0, "origin", -40, [51., 0]), tlm.surfaces.CircularPlane(35.0)),
]

test_surfaces = [s for t, s in test_data]
test_transforms = [t for t, s in test_data]


def demo(rays):
    all_points = torch.empty((0, 2))
    all_normals = torch.empty((0, 2))
    P, V = test_rays[:, 0:2], test_rays[:, 2:4]

    # COLLIDE
    for transform, surface in test_data:
        points, normals = tlm.intersect(surface, P, V, transform)

        if points.numel() > 0:
            all_points = torch.cat((all_points, points), dim=0)
            all_normals = torch.cat((all_normals, normals), dim=0)

    # RENDER
    scene = scene = tlm.viewer.render(points=all_points, normals=all_normals)
    scene["mode"] = "2D"
    scene["camera"] = "XY"

    rays_start = P
    rays_end = P + 100*V
    scene["data"].append(
        tlm.viewer.render_rays(rays_start, rays_end)
    )

    scene["data"].append({"type": "surfaces",
                   "data": [tlm.viewer.render_surface(s, t, dim=2) for t, s in test_data],
                  })

    #pprint(scene)
    
    tlm.viewer.show(scene)
    #tlm.viewer.show(scene, ndigits=3, dump=True)


demo(test_rays)